In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats/test1.zip
/kaggle/input/dogs-vs-cats/train.zip
/kaggle/input/dogs-vs-cats/sampleSubmission.csv


In [4]:
!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

--2022-10-17 20:36:29--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.128, 74.125.26.128, 173.194.215.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M   171MB/s    in 0.4s    

2022-10-17 20:36:30 (171 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [5]:
import zipfile

data = zipfile.ZipFile('./cats_and_dogs_filtered.zip','r')
data.extractall()
data.close()

In [6]:
# check content of train folder
import os
print("Contents of this folder are:")
print(os.listdir('cats_and_dogs_filtered'))

Contents of this folder are:
['validation', 'vectorize.py', 'train']


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale = 1.0/255)
train_data = train_gen.flow_from_directory('cats_and_dogs_filtered/train',target_size = (200,200),class_mode='binary',batch_size = 32)

Found 2000 images belonging to 2 classes.


In [8]:
valid_gen = ImageDataGenerator(rescale = 1.0/255)
valid_data = valid_gen.flow_from_directory('cats_and_dogs_filtered/validation',target_size = (200,200),class_mode='binary',batch_size = 32)

Found 1000 images belonging to 2 classes.


In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
model = Sequential()

2022-10-17 20:36:31.976329: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [10]:
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout
model.add(Conv2D(20,(3,3),activation='relu',input_shape = (200,200,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(40,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(80,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(250,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(150,activation = 'relu'))
model.add(Dense(50,activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(1,activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 20)      560       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 40)        7240      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 40)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 80)        28880     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 80)        0         
_________________________________________________________________
flatten (Flatten)            (None, 42320)             0

In [11]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [12]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
stop = EarlyStopping(patience=20)
check = ModelCheckpoint('best_model.hdf5',monitor='val_loss',save_best_only=True)
model.fit(train_data,epochs=20,callbacks=[stop,check],validation_data=valid_data)

2022-10-17 20:36:32.740535: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20
11/63 [====>.........................] - ETA: 28s - loss: 0.7226 - accuracy: 0.5057

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import random
from tensorflow.keras.preprocessing.image import img_to_array , load_img

successive_output = [layer.output for layer in model.layers]
visual_model = tf.keras.models.Model(inputs = model.input, outputs = successive_output)
img_path = 'cats_and_dogs_filtered/validation/dogs/dog.2140.jpg'
img = load_img(img_path,target_size=(200,200))
x = img_to_array(img)
x = x.reshape((1,) + x.shape)
x/= 255
successive_feature_maps = visual_model.predict(x)

# These are the names of the layers, so you can have them as part of our plot
layer_names = [layer.name for layer in model.layers]

# Display the representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  
  if len(feature_map.shape) == 4:
    
    #-------------------------------------------
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    #-------------------------------------------
    n_features = feature_map.shape[-1]  # number of features in the feature map
    size       = feature_map.shape[ 1]  # feature map shape (1, size, size, n_features)
    
    # Tile the images in this matrix
    display_grid = np.zeros((size, size * n_features))
    
    #-------------------------------------------------
    # Postprocess the feature to be visually palatable
    #-------------------------------------------------
    for i in range(n_features):
      x  = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std ()
      x *=  64
      x += 128
      x  = np.clip(x, 0, 255).astype('uint8')
      display_grid[:, i * size : (i + 1) * size] = x # Tile each filter into a horizontal grid

    #-----------------
    # Display the grid
    #-----------------
    scale = 20. / n_features
    plt.figure( figsize=(scale * n_features, scale) )
    plt.title ( layer_name )
    plt.grid  ( False )
    plt.imshow( display_grid, aspect='auto', cmap='viridis' ) 